##  Assignment Pradeep Reddy

### Import packages

In [45]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [9]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [54]:
session.execute("DROP KEYSPACE IF EXISTS bd23")

In [56]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [57]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS bd23 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [58]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Create a Inv table for testing

In [59]:
session.execute("""
CREATE TABLE IF NOT EXISTS bd23.inv ( 
    sku TEXT, 
    item_name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(sku)
);
""")

### Load data

In [60]:
df = pd.read_csv('inv.csv')
df

,sku,item_name,description,warehouse_num
0,'sku001','bottle','220ml',123
1,'sku002','box','lunchbox',675
2,'sku003','table','woodentable',123
3,'sku004','chair','officechair',878
4,'sku005','belt','leatherbelt',897
5,'sku006','knife','bucherknife',573


In [61]:
for index, row in df.iterrows():
    print(f"sku = {row.sku}, item_name ={row.item_name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

sku = 'sku001', item_name ='bottle', description = '220ml', warehouse_num = 123
sku = 'sku002', item_name ='box', description = 'lunchbox', warehouse_num = 675
sku = 'sku003', item_name ='table', description = 'woodentable', warehouse_num = 123
sku = 'sku004', item_name ='chair', description = 'officechair', warehouse_num = 878
sku = 'sku005', item_name ='belt', description = 'leatherbelt', warehouse_num = 897
sku = 'sku006', item_name ='knife', description = 'bucherknife', warehouse_num = 573


In [62]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ({row.sku}, {row.item_name}, {row.description}, {row.warehouse_num});
        """
    )
    session.execute(f"""
        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ({row.sku}, {row.item_name}, {row.description}, {row.warehouse_num});
        """
    )


        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ('sku001', 'bottle', '220ml', 123);
        

        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ('sku002', 'box', 'lunchbox', 675);
        

        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ('sku003', 'table', 'woodentable', 123);
        

        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ('sku004', 'chair', 'officechair', 878);
        

        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ('sku005', 'belt', 'leatherbelt', 897);
        

        INSERT INTO bd23.inv (sku, item_name, description, warehouse_num)     
        VALUES ('sku006', 'knife', 'bucherknife', 573);
        


In [64]:
rows = session.execute("select (sku, item_name, description, warehouse_num) from bd23.inv where warehouse_num = 123 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

sku003, table, woodentable, 123
sku001, bottle, 220ml, 123
